In [1]:
import sys
sys.path.append('/home/bij/Projects/fdtd/')
import fdtd
import fdtd.backend as bd
import matplotlib.pyplot as plt
from PIL import Image
import torch
import numpy as np

# ## Set Backend
fdtd.set_backend("torch")


# ## Simulation Constants
WAVELENGTH = 1550e-9 # For resolution purposes.
SPEED_LIGHT: float = 299_792_458.0  # [m/s] speed of light

# Frequency of the brainwaves.
BRAIN_FREQ = 3400000000000.0 # Hz


grid = fdtd.Grid(
    #(6.5e-5, 6.5e-5, 6.5e-5,),
    (1.5e-5, 1.5e-5, 1.5e-5,), # Fast version
    #(3.5e-5, 3.5e-5, 3.5e-5,),
    grid_spacing=0.1 * WAVELENGTH,
    permittivity=1.0,
    permeability=1.0,
)

print('Grid Shape: ', grid.shape)



# grid[0, :, :] = fdtd.PeriodicBoundary(name="xbounds")
grid[0:10, :, :] = fdtd.PML(name="pml_xlow")
grid[-10:, :, :] = fdtd.PML(name="pml_xhigh")

# grid[:, 0, :] = fdtd.PeriodicBoundary(name="ybounds")
grid[:, 0:10, :] = fdtd.PML(name="pml_ylow")
grid[:, -10:, :] = fdtd.PML(name="pml_yhigh")

grid[:, :, 0:10] = fdtd.PML(name="pml_zlow")
grid[:, :, -10:] = fdtd.PML(name="pml_zhigh")

grid[40:-40, 40:-40, 40:-40] = fdtd.LearnableAnisotropicObject(permittivity=1.0, is_substrate=False, name="cc_substrate")
qx, qy, qz = np.array(grid.objects[0].inverse_permittivity.shape[0:3])//4
hx, hy, hz = np.array(grid.objects[0].inverse_permittivity.shape[0:3])//2
print(grid.objects[0].inverse_permittivity.shape)

Grid Shape:  (97, 97, 97)
torch.Size([17, 17, 17, 3, 3])


In [2]:
grid[ hx:hx+2, hy:hy+2, hz:hz+2] = fdtd.LineSource(
   period=1.0 / BRAIN_FREQ, name="source", amplitude=0.01,
)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:

yl, xl, zl = grid.objects[0].Ny, grid.objects[0].Nx, grid.objects[0].Nz


#iimg = torch.from_numpy(image) * 100 + 1
iimg = torch.ones_like(grid.objects[0].inverse_permittivity)



print(iimg.shape)
print(qx, qy, qz)
print(hx, hy, hz)

iimg[qx:2*qx, qx:2*qx, :] = 0

iimg = iimg * 100 + 1


grid.objects[0].seed(1.0/iimg)







torch.Size([17, 17, 17, 3, 3])
4 4 4
8 8 8


In [4]:
grid.sources[0].y
grid.E.shape

torch.Size([97, 97, 97, 3])

In [5]:
torch.set_grad_enabled(False)
#grid.visualize(z=hz, animate=True, norm="log")
#vis_steps = 20
vis_steps = 15
step = 0
for i in range(1000000):
    grid.run(vis_steps, progress_bar=False)
    grid.visualize(z=hz, norm='log', animate=True, objcolor=(0, 0, 0, 0), objedgecolor=(1,1,1,0), plot_both_fields=False, save=False, folder='./sim_frames/', index=i, clean_img=True)
    plt.show()
    step += vis_steps
    print('On step: ', step)
x = input('type input to end: ')

AttributeError: 'LineSource' object has no attribute 'profile'